### UC Berkeley, MICS, W202-Cryptography
### Week 02  Breakout 3
### Shamir Secret Sharing

As our lecture notes point out secure multi-party computation is a SUPER HOT area of cybersecurity today!  Shamir Secret Sharing is the leading algorithm in this area.  However, as you will notice, it's not covered in our textbooks, nor in any book I could find.  This makes it a great opportunity for students to learn something that is really hot in industry, but does not have a lot of people with exposure to it!

Sharmir Secret Sharing 
* n users receive a "share" (not the secret)
* a quorum q <= n of the n users, each with a share, are needed to recover the secret

For example, we may have the combination to a safe that needs to be kept secret at a big box store.  We have 12 management employees that each have their own share. We require that 3 of these management employees present their shares to recover the secret combination to the safe.  Any 3 of the 12 can present their share and the secret can be recovered.

The simplified basic principle works like this:  In high school algebra, you learned that 2 points determine a line. If we have two points, we can write an equation for a line. A line is a polynomial of degree 1. 

Generalizing this concept to all polynomials: if we want a quorum of q, we write a polynomial f(x) of degree (q-1).  f(0) will be the secret. Shares can be f(1), f(2), ..., f(n). It will take q shares to recover the secret.

Further, we limit our polynomials to Finite Fields (aka Galios Fields) in modulo prime, usually writen GF(p)

* f(x) = a_(q-1) * x^(q-1) + ... + a_1 * x + a_0 (mod p)
* f(0) = a_0 = secret
* shares: f(1), f(2), f(3)..., f(p-1)

When Shamir secret sharing is used for secure multi-party computation, the minimum size of p is determined based on the size of inputs and operations that are performed during the computation. It could be as small as 100 bits or as big as 4k bits.

Below we will run the example given to us in the lecture.

### Example

We require a quorum of 3 shares to recover the secret, q = 3

We write a polynomial of degree 2 using Shamir's forumla, (q-1) = 2

To create our GF(p), we decide to use p = 97, that is our polynomial will be in (mod 97)

We give out the following shares:
* f(1) = 53
* f(3) = 5
* f(4) = 4

Recover the secret.

In [1]:
from sage.all import *

In [2]:
def my_shamir_recover_secret(q, p, points):
    "given the quorum q, the mod p, and a list of points, recover a shamir secret"
    
    if len(points) != q:
        print ("must submit exactly", q, "points!")
        return
    
    print ("\n")
    
    temp_matrix = []
    
    temp_vector = []
    
    for i in range(0,q):
        
        s1 = "f(" + str(points[i][0]) + ") = "
        s2 = "f(" + str(points[i][0]) + ") = "
        
        temp_row = []
        
        for j in range(q-1,-1,-1):
            
            temp_row.append(points[i][0] ** j)
            
            s1 += "a_" + str(j) + " * " + str(points[i][0] ** j)
            s2 += "a_" + str(j) + " * " + str(points[i][0]) + "^" + str(j)
            
            if j != 0:
                s1 += " + "
                s2 += " + "
                
        temp_matrix.append(temp_row)
        temp_vector.append(points[i][1])
        
        print (s1 + " = " + str(points[i][1]))
        print (s2 + " = " + str(points[i][1]) + "\n")
        
    print ("Place the coefficients from the above system of equations into a matrix of coefficienct and a vector of function values and use a Leibniz solver over GF(p) to find the solution\n")
     
    # create GF(p) to ensure the Liebniz solver will be able to use it throughout,
    # otherwise, if it can't cut by mod during certain steps, it will give the wrong answer
    
    R = IntegerModRing(p)
    
    my_matrix = Matrix(R, temp_matrix)
    print (my_matrix, "\n")
    
    my_vector = vector(R, temp_vector)
    print (my_vector, "\n")
    
    solution = my_matrix.solve_right(my_vector)
    
    print ("solution vector:")
    print (solution, "\n")
            
    # print ("secret:\nf(0) = " + str(Integer(round(solution[-1][0]))))
    print ("secret:\nf(0) = " + str(solution[q-1]))
    
    return solution

In [3]:
solution = my_shamir_recover_secret(3, 97, [(1,53),(3,5),(4,4)])



f(1) = a_2 * 1 + a_1 * 1 + a_0 * 1 = 53
f(1) = a_2 * 1^2 + a_1 * 1^1 + a_0 * 1^0 = 53

f(3) = a_2 * 9 + a_1 * 3 + a_0 * 1 = 5
f(3) = a_2 * 3^2 + a_1 * 3^1 + a_0 * 3^0 = 5

f(4) = a_2 * 16 + a_1 * 4 + a_0 * 1 = 4
f(4) = a_2 * 4^2 + a_1 * 4^1 + a_0 * 4^0 = 4

Place the coefficients from the above system of equations into a matrix of coefficienct and a vector of function values and use a Leibniz solver over GF(p) to find the solution

[ 1  1  1]
[ 9  3  1]
[16  4  1] 

(53, 5, 4) 

solution vector:
(40, 10, 3) 

secret:
f(0) = 3


### Now that we have the solution vector which includes the secret, let's use the solution vector to enumerate all of the values f(0) through f(p-1)

In [4]:
solution

(40, 10, 3)

In [5]:
def my_shamir_list_values(q, p, solution):
    "given a mod value p and a solution vector, f(0) through f(m-1) inclusive"
    
    print ("")
    
    if q != len(solution):
        print ("must submit a solution of ", q, "values")
    
    if p < 100:
        range_limit = p
    else:
        range_limit = 101
    
    for i in range(0,range_limit):
        
        value = 0
        exponent = 0
        
        for j in range(q-1,-1,-1):
           
            value += (i ** exponent) * solution[j]
            exponent += 1
            
        value %= p   
        
        print ("f(" + str(i) + ") = " + str(value))
            
    if i >= 100:
        print ("\n*** only printed the first 100 shares ***")
        

In [6]:
my_shamir_list_values(3, 97, solution)


f(0) = 3
f(1) = 53
f(2) = 86
f(3) = 5
f(4) = 4
f(5) = 83
f(6) = 48
f(7) = 93
f(8) = 24
f(9) = 35
f(10) = 29
f(11) = 6
f(12) = 63
f(13) = 6
f(14) = 29
f(15) = 35
f(16) = 24
f(17) = 93
f(18) = 48
f(19) = 83
f(20) = 4
f(21) = 5
f(22) = 86
f(23) = 53
f(24) = 3
f(25) = 33
f(26) = 46
f(27) = 42
f(28) = 21
f(29) = 80
f(30) = 25
f(31) = 50
f(32) = 58
f(33) = 49
f(34) = 23
f(35) = 77
f(36) = 17
f(37) = 37
f(38) = 40
f(39) = 26
f(40) = 92
f(41) = 44
f(42) = 76
f(43) = 91
f(44) = 89
f(45) = 70
f(46) = 34
f(47) = 78
f(48) = 8
f(49) = 18
f(50) = 11
f(51) = 84
f(52) = 43
f(53) = 82
f(54) = 7
f(55) = 12
f(56) = 0
f(57) = 68
f(58) = 22
f(59) = 56
f(60) = 73
f(61) = 73
f(62) = 56
f(63) = 22
f(64) = 68
f(65) = 0
f(66) = 12
f(67) = 7
f(68) = 82
f(69) = 43
f(70) = 84
f(71) = 11
f(72) = 18
f(73) = 8
f(74) = 78
f(75) = 34
f(76) = 70
f(77) = 89
f(78) = 91
f(79) = 76
f(80) = 44
f(81) = 92
f(82) = 26
f(83) = 40
f(84) = 37
f(85) = 17
f(86) = 77
f(87) = 23
f(88) = 49
f(89) = 58
f(90) = 50
f(91) = 25
f(92) = 80


In [7]:
# try various combinations of 3 points to verify that they all give the same solution

solution = my_shamir_recover_secret(3, 97, [(43,91),(67,7),(96,33)])



f(43) = a_2 * 1849 + a_1 * 43 + a_0 * 1 = 91
f(43) = a_2 * 43^2 + a_1 * 43^1 + a_0 * 43^0 = 91

f(67) = a_2 * 4489 + a_1 * 67 + a_0 * 1 = 7
f(67) = a_2 * 67^2 + a_1 * 67^1 + a_0 * 67^0 = 7

f(96) = a_2 * 9216 + a_1 * 96 + a_0 * 1 = 33
f(96) = a_2 * 96^2 + a_1 * 96^1 + a_0 * 96^0 = 33

Place the coefficients from the above system of equations into a matrix of coefficienct and a vector of function values and use a Leibniz solver over GF(p) to find the solution

[ 6 43  1]
[27 67  1]
[ 1 96  1] 

(91, 7, 33) 

solution vector:
(40, 10, 3) 

secret:
f(0) = 3


In [8]:
solution = my_shamir_recover_secret(3, 97, [(66,12),(27,42),(11,6)])



f(66) = a_2 * 4356 + a_1 * 66 + a_0 * 1 = 12
f(66) = a_2 * 66^2 + a_1 * 66^1 + a_0 * 66^0 = 12

f(27) = a_2 * 729 + a_1 * 27 + a_0 * 1 = 42
f(27) = a_2 * 27^2 + a_1 * 27^1 + a_0 * 27^0 = 42

f(11) = a_2 * 121 + a_1 * 11 + a_0 * 1 = 6
f(11) = a_2 * 11^2 + a_1 * 11^1 + a_0 * 11^0 = 6

Place the coefficients from the above system of equations into a matrix of coefficienct and a vector of function values and use a Leibniz solver over GF(p) to find the solution

[88 66  1]
[50 27  1]
[24 11  1] 

(12, 42, 6) 

solution vector:
(40, 10, 3) 

secret:
f(0) = 3


In [9]:
solution = my_shamir_recover_secret(3, 97, [(12,63),(10,29),(9,35)])



f(12) = a_2 * 144 + a_1 * 12 + a_0 * 1 = 63
f(12) = a_2 * 12^2 + a_1 * 12^1 + a_0 * 12^0 = 63

f(10) = a_2 * 100 + a_1 * 10 + a_0 * 1 = 29
f(10) = a_2 * 10^2 + a_1 * 10^1 + a_0 * 10^0 = 29

f(9) = a_2 * 81 + a_1 * 9 + a_0 * 1 = 35
f(9) = a_2 * 9^2 + a_1 * 9^1 + a_0 * 9^0 = 35

Place the coefficients from the above system of equations into a matrix of coefficienct and a vector of function values and use a Leibniz solver over GF(p) to find the solution

[47 12  1]
[ 3 10  1]
[81  9  1] 

(63, 29, 35) 

solution vector:
(40, 10, 3) 

secret:
f(0) = 3


### Now let's consider that one holder of a share decides to attack the algorithm. 

If one holder of a share decided to give the wrong number to the share, what happens?

The other users will recover an invalid number for the secret.

Worse, the holder who gave the wrong number, if he received the other shares submitted, can recover the real secret.

In [10]:
# change the valid point (9,35) to the invalid point (9,34)
# it will recover an invalid number for the secret

solution = my_shamir_recover_secret(3, 97, [(12,63),(10,29),(9,34)])



f(12) = a_2 * 144 + a_1 * 12 + a_0 * 1 = 63
f(12) = a_2 * 12^2 + a_1 * 12^1 + a_0 * 12^0 = 63

f(10) = a_2 * 100 + a_1 * 10 + a_0 * 1 = 29
f(10) = a_2 * 10^2 + a_1 * 10^1 + a_0 * 10^0 = 29

f(9) = a_2 * 81 + a_1 * 9 + a_0 * 1 = 34
f(9) = a_2 * 9^2 + a_1 * 9^1 + a_0 * 9^0 = 34

Place the coefficients from the above system of equations into a matrix of coefficienct and a vector of function values and use a Leibniz solver over GF(p) to find the solution

[47 12  1]
[ 3 10  1]
[81  9  1] 

(63, 29, 34) 

solution vector:
(72, 82, 60) 

secret:
f(0) = 60


### Let's create our own share with a polynomial of degree 4, with a quorum q of 5, GF(p) with p of 97, a secret of 15, and 4 random values for the remaining a's

In [11]:
my_shamir_list_values(5, 97, [32,41,14,13,15])


f(0) = 15
f(1) = 18
f(2) = 64
f(3) = 96
f(4) = 49
f(5) = 44
f(6) = 0
f(7) = 22
f(8) = 13
f(9) = 62
f(10) = 56
f(11) = 68
f(12) = 66
f(13) = 10
f(14) = 46
f(15) = 21
f(16) = 65
f(17) = 9
f(18) = 64
f(19) = 45
f(20) = 50
f(21) = 72
f(22) = 96
f(23) = 2
f(24) = 50
f(25) = 7
f(26) = 20
f(27) = 34
f(28) = 83
f(29) = 96
f(30) = 91
f(31) = 78
f(32) = 59
f(33) = 28
f(34) = 68
f(35) = 60
f(36) = 71
f(37) = 63
f(38) = 87
f(39) = 89
f(40) = 7
f(41) = 62
f(42) = 79
f(43) = 69
f(44) = 35
f(45) = 69
f(46) = 61
f(47) = 87
f(48) = 21
f(49) = 20
f(50) = 39
f(51) = 25
f(52) = 14
f(53) = 34
f(54) = 8
f(55) = 45
f(56) = 52
f(57) = 25
f(58) = 49
f(59) = 7
f(60) = 65
f(61) = 90
f(62) = 38
f(63) = 51
f(64) = 69
f(65) = 24
f(66) = 34
f(67) = 15
f(68) = 69
f(69) = 96
f(70) = 85
f(71) = 17
f(72) = 59
f(73) = 79
f(74) = 34
f(75) = 67
f(76) = 22
f(77) = 26
f(78) = 4
f(79) = 67
f(80) = 27
f(81) = 76
f(82) = 10
f(83) = 5
f(84) = 35
f(85) = 66
f(86) = 56
f(87) = 52
f(88) = 93
f(89) = 16
f(90) = 38
f(91) = 77
f(92) 

In [12]:
# let's verify by solving based on quorum q of 5, GF(p) with p of 97, and 5 points from the previous printout

solution = my_shamir_recover_secret(5, 97, [(13,10),(16,65),(27,34),(47,87),(78,4)])



f(13) = a_4 * 28561 + a_3 * 2197 + a_2 * 169 + a_1 * 13 + a_0 * 1 = 10
f(13) = a_4 * 13^4 + a_3 * 13^3 + a_2 * 13^2 + a_1 * 13^1 + a_0 * 13^0 = 10

f(16) = a_4 * 65536 + a_3 * 4096 + a_2 * 256 + a_1 * 16 + a_0 * 1 = 65
f(16) = a_4 * 16^4 + a_3 * 16^3 + a_2 * 16^2 + a_1 * 16^1 + a_0 * 16^0 = 65

f(27) = a_4 * 531441 + a_3 * 19683 + a_2 * 729 + a_1 * 27 + a_0 * 1 = 34
f(27) = a_4 * 27^4 + a_3 * 27^3 + a_2 * 27^2 + a_1 * 27^1 + a_0 * 27^0 = 34

f(47) = a_4 * 4879681 + a_3 * 103823 + a_2 * 2209 + a_1 * 47 + a_0 * 1 = 87
f(47) = a_4 * 47^4 + a_3 * 47^3 + a_2 * 47^2 + a_1 * 47^1 + a_0 * 47^0 = 87

f(78) = a_4 * 37015056 + a_3 * 474552 + a_2 * 6084 + a_1 * 78 + a_0 * 1 = 4
f(78) = a_4 * 78^4 + a_3 * 78^3 + a_2 * 78^2 + a_1 * 78^1 + a_0 * 78^0 = 4

Place the coefficients from the above system of equations into a matrix of coefficienct and a vector of function values and use a Leibniz solver over GF(p) to find the solution



[43 63 72 13  1]
[61 22 62 16  1]
[75 89 50 27  1]
[96 33 75 47  1]
[50 28 70 78  1] 

(10, 65, 34, 87, 4) 

solution vector:
(32, 41, 14, 13, 15) 

secret:
f(0) = 15


### We have been using a low value for p for testing purposes.  Let's try it with larger values for p

In [13]:
def my_shamir_given_bit_size(b, q, secret):
    "given a bit size, a quorum size, and a secret, create a list of shares"


    upper_limit = (2^b) - 1
    lower_limit = (2^(b-1))
    
    p = random_prime(upper_limit, false, lower_limit)
    
    print ("\nquorum:", q, "\n")
    
    print ("p:", p, "\n")
    
    # let's generate q-1 random numbers for the solution vector
    solution = []
    for i in range(0,q-1):
        solution.append(randint(floor(p/2), floor(p*0.9)))
    
    # the secret will b the last spot in the solution vector
    solution.append(secret)
    
    print ("solution:", solution, "\n")
    
    my_shamir_list_values(q, p, solution)

In [14]:
#set the number of bits for p
b = 64
#b = 128
#b = 512
#b = 1024
#b = 2048
#b = 4096

# set the quorum
q = 5

#set the secret (my high school locker combination from 1979 - 1983)
secret = 192935

my_shamir_given_bit_size(b, q, secret)


quorum: 5 

p: 12448885587372983053 

solution: [9202842298751828775, 10906939795474264502, 6837100599868031071, 9042783072982306872, 192935] 


f(0) = 192935
f(1) = 11091894592330658049
f(2) = 6059480767054680213
f(3) = 8180152430894190749
f(4) = 172921325782367466
f(5) = 11340616140474516031
f(6) = 5529912224191222492
f(7) = 5620187460348183808
f(8) = 6381323154945326478
f(9) = 11820360798685755700
f(10) = 10283730892229789265
f(11) = 11803909708313906716
f(12) = 10752762529631800189
f(13) = 10739314823580340519
f(14) = 12160866654886594187
f(15) = 12202992685607823320
f(16) = 4839542175131485691
f(17) = 5730410154921200825
f(18) = 2425995079024817787
f(19) = 4162741173566347394
f(20) = 4516481674627013056
f(21) = 6300210002991216882
f(22) = 9115194176773556627
f(23) = 9350976811418825692
f(24) = 185375119702013124
f(25) = 8931137673847252775
f(26) = 9893745293916941931
f(27) = 11513610159422622683
f(28) = 6570533459833049715
f(29) = 9530362156693139463
f(30) = 6749446634132037903
f

In [15]:
# now let's recover the secret.  I put in fixed points from a run I made with 64 bit.  you can copy this cell and fill in your values

solution = my_shamir_recover_secret(5, 18429518054934476701, [(1,8898586958560387597),(33,10187478313697365727),(56,16661803173988792227),(77,10127357201381662851),(96,1919136716310013690)])



f(1) = a_4 * 1 + a_3 * 1 + a_2 * 1 + a_1 * 1 + a_0 * 1 = 8898586958560387597
f(1) = a_4 * 1^4 + a_3 * 1^3 + a_2 * 1^2 + a_1 * 1^1 + a_0 * 1^0 = 8898586958560387597

f(33) = a_4 * 1185921 + a_3 * 35937 + a_2 * 1089 + a_1 * 33 + a_0 * 1 = 10187478313697365727
f(33) = a_4 * 33^4 + a_3 * 33^3 + a_2 * 33^2 + a_1 * 33^1 + a_0 * 33^0 = 10187478313697365727

f(56) = a_4 * 9834496 + a_3 * 175616 + a_2 * 3136 + a_1 * 56 + a_0 * 1 = 16661803173988792227
f(56) = a_4 * 56^4 + a_3 * 56^3 + a_2 * 56^2 + a_1 * 56^1 + a_0 * 56^0 = 16661803173988792227

f(77) = a_4 * 35153041 + a_3 * 456533 + a_2 * 5929 + a_1 * 77 + a_0 * 1 = 10127357201381662851
f(77) = a_4 * 77^4 + a_3 * 77^3 + a_2 * 77^2 + a_1 * 77^1 + a_0 * 77^0 = 10127357201381662851

f(96) = a_4 * 84934656 + a_3 * 884736 + a_2 * 9216 + a_1 * 96 + a_0 * 1 = 1919136716310013690
f(96) = a_4 * 96^4 + a_3 * 96^3 + a_2 * 96^2 + a_1 * 96^1 + a_0 * 96^0 = 1919136716310013690

Place the coefficients from the above system of equations into a matrix of coe

### Discussion (Time Permitting)

Discuss applications for this technology. As mentioned before, it's super hot in industry right now.